# Entrenamiento de modelos encuestadores de hogares EPH

In [1]:
# Convertir a script de python con nbconvert, 
# removiendo las celdas que estan pensadas para la notebook interactiva, no para el script.
# De esta forma no vamos a incluir las celdas de input, 
# 

# jupyter nbconvert --to python --TagRemovePreprocessor.remove_cell_tags "interactive-only"  02\ -\ Train\ ML\ algos\ on\ EPH.ipynb

## Funciona en las versiones mas nuevas de jupyter, en caso que nbconvert de error, verificar:
# jupyter --version

## Correr:
# conda update jupyter

# Y volver a intentar.

In [2]:
import sklearn

print("scikit-learn version:", sklearn.__version__)

scikit-learn version: 1.3.0


### Seteo de Parametros

In [3]:
from IPython import get_ipython

if get_ipython() is None:
    print('ARGUMENTOS TOMADOS DE CLI')
    import argparse

    parser = argparse.ArgumentParser(description='A script to process data for a range of years')

    parser.add_argument('-y','--years', nargs='+', help='Set the range of years to process data for. Default is the current year and the next year', required=False, type=int, default=[2022, 2023])
    parser.add_argument('-ow','--overwrite', nargs=1, required=False, default= True, help='Flag to specify if previous data should be overwritten. Default is True')

    args = parser.parse_args()

    overwrite = args.overwrite
    startyr = args.years[0]
    endyr = args.years[1]
    
else:
    print('ARGUMENTOS INTRODUCIDOS POR EL USUARIO')
    startyr = input("Enter the start year [default: 2022]: ") or 2022
    endyr = input("Enter the end year [default: 2023]: ") or 2023
    overwrite = input("Do you want to overwrite previous data? [y/n] [default: y]: ") or "y"

    if overwrite.lower() == "y":
        overwrite = True
    else:
        overwrite = False

    #Convert the input to integers
    startyr = int(startyr)
    endyr = int(endyr)

    print("Start year: ", startyr)
    print("End year: ", endyr)
    print("Overwrite: ", overwrite)


ARGUMENTOS INTRODUCIDOS POR EL USUARIO
Start year:  2003
End year:  2015
Overwrite:  True


# Training

In [4]:
# Column names
# y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
#           'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
#           'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']

# quitando las columns ['PP10E', 'PP10D', 'CH16', 'PP10C', 'CH08'] (changas, hace cuanto no trabaja, se mudo en 5 anios, nunca trabajo_)
# y_cols = ['CAT_OCUP', 'P47T', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'T_VI', 
#           'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 
#           'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']

x_cols1 = ['IX_TOT', 'P02', 'P03', 'AGLO_rk', 'Reg_rk', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']

x_cols2 = x_cols1 + predecir1
predecir2 = ['INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']

x_cols3 = x_cols2 + predecir2
# La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
# predecir3 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
predecir3 = ['PP07G1','PP07G_59', 'PP07I', 'PP07J', 'PP07K']

# Columnas de ingresos. Necesitan una regresion...
columnas_pesos = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

x_cols4 = x_cols3 + predecir3
# Columnas de ingresos. Necesitan una regresion...

## Funciones clasificador y regresor

In [5]:
### Save models at: (ocupan bastante memoria en disco)
# models_path = '/media/miglesia/Elements/suite/encuestador-de-hogares'
models_path = './..'


In [6]:
# import sys
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

In [7]:
# Si el modulo sklearn no esta instalado, lo instalo
try:
    import sklearn
except ImportError:
    !pip install sklearn
    import sklearn

In [8]:
from sklearn.model_selection import train_test_split # importamos las funciones para dividir los datos
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # importamos los modelos de random forest
import os # para trabajar con el sistema operativo
import joblib # para guardar el modelo entrenado

def fit_model(train_data, x_cols, y_cols, out_filename, model):
    """
    Entrena un modelo de random forest.
    train_data: dataframe con los datos de entrenamiento
    x_cols: lista con los nombres de las columnas de entrada del modelo
    y_cols: lista con los nombres de las columnas de salida del modelo
    out_filename: string con la ruta y nombre del archivo donde se guardará el modelo
    model: instancia del modelo a entrenar
    """
    X = train_data[x_cols] # separamos las columnas de entrada
    y = train_data[y_cols] # separamos las columnas de salida
    
    X, X_test, y, y_test = train_test_split(X, y, test_size=0.1) # dividimos los datos en entrenamiento y test
    
    clf = model.fit(X.values, y.values) # entrenamos el modelo
    
    # guardamos el modelo en disco
    if not os.path.exists(models_path + '/fitted_RF/'):
        os.makedirs(models_path + '/fitted_RF/')
    joblib.dump(model, out_filename, compress=3)
    print('saved model at: ' + out_filename)



#     print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])
    del clf
    del X; del y # liberar memoria eliminando los dataframes mas pesados

In [10]:
import pandas as pd

In [13]:
# train_data = pd.read_csv('./../data/training/EPHARG_train_23.csv')

# train_q = train_data.loc[train_data.Q == q]
# # fit_model(train_q, x_cols = x_cols4, y_cols = y_cols4, out_filename = out,
# #         model = RandomForestRegressor(n_estimators=1, max_depth = 20, n_jobs = -1))

# model = RandomForestRegressor(n_estimators=1, max_depth = 20, n_jobs = -1)
# y_cols = columnas_pesos
# x_cols = x_cols4

# X = train_data[x_cols] # separamos las columnas de entrada
# y = train_data[y_cols] # separamos las columnas de salida

# X, X_test, y, y_test = train_test_split(X, y, test_size=0.1) # dividimos los datos en entrenamiento y test

# clf = model.fit(X.values, y.values) # entrenamos el modelo

# # guardamos el modelo en disco
# if not os.path.exists(models_path + '/fitted_RF/'):
#     os.makedirs(models_path + '/fitted_RF/')

# # out_filename = models_path + '/fitted_RF/' + out + '.joblib'
# out_filename = './../fitted_RF/clf4_2023-02-15_ARG'
# joblib.dump(model, out_filename, compress=3)
# print('saved model at: ' + out_filename)



## Loop principal. Entrenar y guardar modelos

In [14]:
import pandas as pd
from numpy import log10

In [16]:
# pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')

# './../data/training/EPHARG_train_'+yr[2:]+'.csv'

In [18]:
# ## Entrenar modelo, para cada trimestre
# for q in train_data.Q.unique():
#     print(q)
#     out = models_path + '/fitted_RF/clf4_'+q+'_ARG'
#     if (not os.path.exists(out)) or (overwrite):
#         train_q = train_data.loc[train_data.Q == q]
#         fit_model(train_q, x_cols = x_cols4, y_cols = columnas_pesos, out_filename = out,
#                     model = RandomForestRegressor(n_estimators=1, max_depth = 20, n_jobs = -1))
#         del train_q;

In [19]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train_data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    
    ## ETAPA 1:
    out = models_path + '/fitted_RF/clf1_'+yr+'_ARG'
    if (not os.path.exists(out)) or (overwrite):
        fit_model(train_data, x_cols = x_cols1, y_cols = predecir1, out_filename = out,
                 model = RandomForestClassifier(n_estimators=100, max_depth = 15, n_jobs = -1))
    
    ## ETAPA 2:
    out = models_path + '/fitted_RF/clf2_'+yr+'_ARG'
    if (not os.path.exists(out)) or (overwrite):
        fit_model(train_data, x_cols = x_cols2, y_cols = predecir2, out_filename = out,
                 model = RandomForestClassifier(n_estimators=100, max_depth = 15, n_jobs = -1))
    
    ## ETAPA 3:
    out = models_path + '/fitted_RF/clf3_'+yr+'_ARG'
    if (not os.path.exists(out)) or (overwrite):
        fit_model(train_data, x_cols = x_cols3, y_cols = predecir3, out_filename = out,
                 model = RandomForestClassifier(n_estimators=100, max_depth = 15, n_jobs = -1))
    
    ## ETAPA 4 (Regresion)
    ## Tomar log de las columnas en pesos.
    train_data[columnas_pesos] = log10(train_data[columnas_pesos].clip(-.9) + 1)

    ## Entrenar modelo, para cada trimestre
    for q in train_data.Q.unique():
        print(q)
        out = models_path + '/fitted_RF/clf4_'+q+'_ARG'
        if (not os.path.exists(out)) or (overwrite):
            train_q = train_data.loc[train_data.Q == q]
            fit_model(train_q, x_cols = x_cols4, y_cols = columnas_pesos, out_filename = out,
                     model = RandomForestRegressor(n_estimators=1, max_depth = 20, n_jobs = -1))
            del train_q;

    del train_data; 

2003
saved model at: ./../fitted_RF/clf1_2003_ARG
saved model at: ./../fitted_RF/clf2_2003_ARG
saved model at: ./../fitted_RF/clf3_2003_ARG
2003-08-15
saved model at: ./../fitted_RF/clf4_2003-08-15_ARG
2003-11-15
saved model at: ./../fitted_RF/clf4_2003-11-15_ARG
2004
saved model at: ./../fitted_RF/clf1_2004_ARG
saved model at: ./../fitted_RF/clf2_2004_ARG
saved model at: ./../fitted_RF/clf3_2004_ARG
2004-05-15
saved model at: ./../fitted_RF/clf4_2004-05-15_ARG
2004-08-15
saved model at: ./../fitted_RF/clf4_2004-08-15_ARG
2004-02-15
saved model at: ./../fitted_RF/clf4_2004-02-15_ARG
2004-11-15
saved model at: ./../fitted_RF/clf4_2004-11-15_ARG
2005
saved model at: ./../fitted_RF/clf1_2005_ARG
saved model at: ./../fitted_RF/clf2_2005_ARG
saved model at: ./../fitted_RF/clf3_2005_ARG
2005-02-15
saved model at: ./../fitted_RF/clf4_2005-02-15_ARG
2005-05-15
saved model at: ./../fitted_RF/clf4_2005-05-15_ARG
2005-11-15
saved model at: ./../fitted_RF/clf4_2005-11-15_ARG
2005-08-15
saved model